In [16]:
#import libraries
import tensorflow as tf
import numpy as np
import cv2
import copy
import PIL
import pickle
from scipy import stats
import os
import matplotlib.pyplot as plt
################
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model

In [17]:
#load FaceNet model
FaceNet=tf.keras.models.load_model('facenet_keras.h5',compile=False)

#load trained YOLO model
cascade_path = 'haarcascade_frontalface_alt2.xml'

#image size
image_size=160

In [18]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output
def load_and_align_images(filepaths, margin):
    cascade = cv2.CascadeClassifier(cascade_path)
    
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)

        faces = cascade.detectMultiScale(img,
                                         scaleFactor=1.1,
                                         minNeighbors=3)
        (x, y, w, h) = faces[0]
        cropped = img[y-margin//2:y+h+margin//2,
                      x-margin//2:x+w+margin//2, :]
        aligned = resize(cropped, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
            
    return np.array(aligned_images)
def calc_embs(filepaths, margin=10, batch_size=1):
    aligned_images = prewhiten(load_and_align_images(filepaths, margin))
    pd = []
    for start in range(0, len(aligned_images), batch_size):
        pd.append(FaceNet.predict_on_batch(aligned_images[start:start+batch_size]))
    embs = l2_normalize(np.concatenate(pd))

    return embs
def calc_dist(img_name0, img_name1):
    return distance.euclidean(data[img_name0]['emb'], data[img_name1]['emb'])

def calc_dist_plot(img_name0, img_name1):
    print(calc_dist(img_name0, img_name1))
    plt.subplot(1, 2, 1)
    plt.imshow(imread(data[img_name0]['image_filepath']))
    plt.subplot(1, 2, 2)
    plt.imshow(imread(data[img_name1]['image_filepath']))

In [19]:
#load the image pathes
images_path=os.listdir('ref images')

In [20]:
embeddings=calc_embs(os.listdir('ref images'))

In [21]:
#sanitiy check
print(len(embeddings))
print(embeddings[0].shape)

6
(128,)


In [22]:
#creat dictionary with names
ref_embeddings_names={
    0:'Mustafa',
    1:'Wael',
    2:'Fawzy',
    3:'Hesham',
    4:'Sayed',
    5:'Omar'
}

In [23]:
#save the embeddings
np.save('ref_embeddings',embeddings)

with open('ref_embeddings_names.pkl', 'wb') as f:
    pickle.dump(ref_embeddings_names, f)

In [24]:
print(embeddings[0].shape)

(128,)
